<a href="https://colab.research.google.com/github/Trace55/Tweet_generation_classification/blob/main/DS2_Final_Project_git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -U transformers datasets sentencepiece pandas scikit-learn

In [ ]:
import re

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def Convert(string):
    li = list(string.split("_____?_____."))
    return li

def clean_tw_df(df): 
    data = df['tweet'].str.cat(sep= '_____?_____.')
    data = re.sub('(?:\s)@[^, ]*', '', data)
    file_name = re.sub(r'^https?:\/\/.*[\r\n]*', '', data, flags=re.MULTILINE)
    file_name = re.sub(r'http\S+', '', file_name)
    df_txt = deEmojify(file_name)
    df_ls = Convert(df_txt)
    df = pd.DataFrame(df_ls, columns = ['tweet'])
    return df

def clean_tw_to_file(df, name):
    data = df['tweet'].str.cat(sep= '\n')
    data = re.sub('(?:\s)@[^, ]*', '', data)
    file_name = re.sub(r'^https?:\/\/.*[\r\n]*', '', data, flags=re.MULTILINE)
    file_name = re.sub(r'http\S+', '', file_name)
    df_txt = deEmojify(file_name)
    x = file_name
    with open(f'{name}.txt', 'w+') as fh:
        fh.write(x)
    X = f'{name}.txt'
    return X

In [ ]:
import pandas as pd
from google.colab import drive 
drive.mount('/content/drive')

path1= '/content/drive/MyDrive/D.S 2 Project/pro_mask.csv'
p_df = pd.read_csv(path1)

path2= '/content/drive/MyDrive/D.S 2 Project/anti_mask.csv'
a_df = pd.read_csv(path2)

anti_df = clean_tw_df(a_df)
pro_df = clean_tw_df(p_df)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2,4,11,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
import pathlib, shutil, dataclasses, contextlib, torch, numpy as np, pandas as pd
from typing import Any
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, f1_score
from datasets import Dataset, DatasetDict
from transformers import Trainer, TrainingArguments, pipeline, AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification
 
@dataclasses.dataclass
class NLP:
    model_checkpoint: str = 'distilbert-base-uncased'
    model_base:       Any = AutoModelForSequenceClassification
    task:             str = 'text-generation'
    path:             str = '/content/drive/MyDrive/D.S 2 Project/'
    learning_rate:    float = 1e-5
    batch_size:       int = 32
    warmup:           int = 600
    max_seq_length:   int = 128
    num_epochs:       int = 35
    # values copied from https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english

    def __post_init__(self):
        self.path = pathlib.Path(self.path)
        self.path_model = self.path / f'{self.task}-{self.model_checkpoint.split("/")[-1]}'
        self.path_model.mkdir(parents=True, exist_ok=True)
        self.model_config = {'pretrained_model_name_or_path':self.model_checkpoint,
                             'max_length':self.max_seq_length,
                             'task_specific_params': {'text-generation': {'do_sample': True, 'max_length': self.max_seq_length}},}
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_checkpoint, use_fast=True, model_max_length=self.max_seq_length)
        if self.tokenizer._pad_token is None:
            self.tokenizer._pad_token = self.tokenizer._eos_token
 
    def __getitem__(self, key):
        return getattr(self, key)
 
    def __setitem__(self, key, value):
        return setattr(self, key, value)
 
    def _listify(self, x):
        if isinstance(x, list) or isinstance(x, tuple) or isinstance(x, range):
            return list(x)
        else:
            return [x]

    def preprocess(self, text):
        tokens = self.tokenizer(text, truncation=False, padding='max_length', return_tensors='pt')
        T = tokens['input_ids'].shape[1]
        sh = (T // self.max_seq_length, self.max_seq_length)
        T = np.product(sh)
        return {key:val[0,:T].reshape(sh) for key, val in tokens.items()}

    def splitter(self, ds, frac=0.2):
        frac = min(frac, 1-frac)
        idxes = next(StratifiedShuffleSplit(test_size=frac).split(ds['labels'], ds['labels']))
        return [ds.select(idx) for idx in idxes]

    def make_dataset(self, source_data, text_col='text', label_col='label', holdout_frac=0.1, validate_frac=0.2):
        if label_col not in source_data.columns:
            print(f'Label column {label_col} not found - filling with 0')
            source_data[label_col] = 0
        self.df = pd.DataFrame(source_data).dropna(subset=[text_col, label_col]).sample(frac=1.0).reset_index(drop=True)

        tokens = dict()
        for label, text in self.df[[label_col, text_col]].values:
            tensors = self.preprocess(text)
            tensors['labels'] = [label for _ in range(tensors['input_ids'].shape[0])]
            for key, val in tensors.items():
                try:
                    tokens[key].append(val)
                except:
                    tokens[key] = [val]
        L = np.hstack(tokens.pop('labels'))
        id2label = dict(enumerate(np.unique(L)))
        label2id = {l:i for i,l in id2label.items()}
        tokens = {key: torch.row_stack(ls) for key, ls in tokens.items()}
        tokens['labels'] = [label2id[l] for l in L]
        self.model_config['id2label'] = id2label
        self.model_config['label2id'] = label2id

        D = DatasetDict({'train': Dataset.from_dict(tokens)})
        for nm, frac in (('holdout',holdout_frac), ('validate',validate_frac)):
            if frac > 0 and frac <=1:
                D['train'], D[nm] = self.splitter(D['train'], frac)
                
        self.dataset = D
        if self.task == 'text-generation':
            def f(X):
                X['labels'] = X['input_ids']
                return X
            self.dataset = D.map(f)
        return self.dataset

    def make_model(self, overwrite=False):
        if overwrite:
            shutil.rmtree(self.path_model, ignore_errors=True)

        with contextlib.suppress(OSError):
            self.model = self.model_base.from_pretrained(self.path_model)
            print(f'Using existing model at {str(self.path_model)}')

        if not hasattr(self, 'model'):
            print(f'No model found at {str(self.path_model)} - creating new one')
            assert hasattr(self, 'dataset'), 'No dataset found - please run make_dataset first'
            self.train(self.dataset, overwrite)
            self.model.save_pretrained(self.path_model)

        self.model = self.model.to('cpu')
        self.pipe = pipeline(task=self.task, model=self.model, tokenizer=self.tokenizer)

    def tokenize(self, text):
        tokens = self.tokenizer(text, truncation=False, return_tensors='pt')
        s = self.max_seq_length
        prefix = {key:val[:,:-s] for key, val in tokens.items()}
        suffix = {key:val[:,-s:] for key, val in tokens.items()}
        return tokens, prefix, suffix

    def generate(self, prompts="", max_new_tokens=20, **kwargs):
        prompts = self._listify(prompts)
        outputs = []
        for text in prompts:
            tokens, prefix, suffix = self.tokenize(text)
            t = tokens['input_ids'].shape[1]
            if t > self.max_seq_length:
                print(f'num_tokens {t} > model max_seq_length {self.max_seq_length}; using the tail for prompt')
            L = max_new_tokens+suffix['input_ids'].shape[1]
            gen = self.model.generate(**suffix, max_length=L, **kwargs)
            o = torch.cat((prefix['input_ids'],gen), axis=1)
            d = self.tokenizer.decode(o[0])
            outputs.append(d)
        return outputs if len(outputs) > 1 else outputs[0]

    def train(self, dataset=None, overwrite=False):
        if dataset is None:
            dataset = self.dataset
        self.path_model.mkdir(parents=True, exist_ok=True)

        training_args = TrainingArguments(
            output_dir                  = self.path_model,
            overwrite_output_dir        = overwrite,
            evaluation_strategy         = 'epoch',
            logging_strategy            = 'no',
            logging_dir                 = self.path_model / 'logs',
            save_strategy               = 'no',
            load_best_model_at_end      = False,
            learning_rate               = self.learning_rate,
            per_device_train_batch_size = self.batch_size,
            per_device_eval_batch_size  = self.batch_size,
            warmup_steps                = self.warmup,
            num_train_epochs            = self.num_epochs,
        )

        self.model = self.model_base.from_pretrained(**self.model_config)
        trainer = Trainer(
            model=self.model,
            args =training_args,
            train_dataset=dataset['train'],
            eval_dataset =dataset['validate'],)
        trainer.train()

base_path = '/content/drive/MyDrive/D.S 2 Project/'
data_kwargs  = {'text_col'        : 'tweet',
                'label_col'       : 'view',
                'holdout_frac'    : 0.0,
                'validate_frac'   : 0.2,}

In [ ]:
from sklearn.model_selection import train_test_split

pro_df = pro_df.sample(anti_df.shape[0],random_state =41) #make same sizes
pro_df['view'] = 'pro'
anti_df['view'] = 'anti'
all_data = pd.concat([pro_df,anti_df])
all_data = all_data.sample(frac=1).reset_index(drop=True)
df_train, df_hold = train_test_split(all_data, test_size=0.01, random_state= 41)
source_data = df_train

In [ ]:
# Text generation
model_kwargs = {'model_checkpoint': 'distilgpt2',
                'model_base'      : AutoModelForCausalLM,
                'task'            : 'text-generation',
                # can adjust training parameters here such as num_epochs & batch size - see first lines of class NLP definition
                }
gens = dict()
for g, df in source_data.groupby('view'):
    gen = NLP(**model_kwargs, path=f'{base_path}/{g}')
    gen.make_dataset(df, **data_kwargs)
    gen.make_model()
    gens[g] = gen

In [ ]:
## Text-Classifier

model_kwargs = {'model_checkpoint': 'distilbert-base-uncased',
                'model_base'      : AutoModelForSequenceClassification,
                'task'            : 'sentiment-analysis',
                # can adjust training parameters here such as num_epochs & batch size - see first lines of class NLP definition
                }
clf = NLP(**model_kwargs, path=base_path)
clf.make_dataset(source_data, **data_kwargs)
clf.make_model()

Using existing model at /content/drive/MyDrive/D.S 2 Project/sentiment-analysis-distilbert-base-uncased


In [ ]:
gen_kwargs = {'repetition_penalty':10.0,  # penalty for repeating yourself
              'max_new_tokens':10,  # max tokens to add for each call to generate

              }

min_words = 50  # stop when this many words are in text

dff = pd.DataFrame()

for j in range(df_hold.shape[0]):
    for g,gen in gens.items(): 
        text = " ".join(df_hold['tweet'].iloc[j].split()[:8])
        for i in range(10):  # run generate up to this many times to try to get min_words
            text = gen.generate(text, **gen_kwargs)  # call generate
            L = len(text.split(" "))  # compute number of words by splitting on spaces
            print(f'{i,L}\n{text}\n\n')
            if L >= min_words:  # stop if enough words
                ls = [text]
                ls.append(g)
                ls = [ls]
                print(ls)
                dff1 = pd.DataFrame(ls, columns = ['tweet','view'])
                dff = pd.concat([dff1,dff])
                break
        clf.pipe(text)  # classify

In [ ]:
gen_kwargs = {'repetition_penalty':10.0,  # penalty for repeating yourself
              'max_new_tokens':10,  # max tokens to add for each call to generate

              }

min_words = 50 
dff = pd.DataFrame()
for k in range(700):
    for g,gen in gens.items(): 
        text = " "
        for i in range(10):  # run generate up to this many times to try to get min_words
            text = gen.generate(text, **gen_kwargs)  # call generate
            L = len(text.split(" "))  # compute number of words by splitting on spaces
            print(f'{i,L}\n{text}\n\n')
            if L >= min_words:  # stop if enough words
                ls = [text]
                ls.append(g)
                ls = [ls]
                print(ls)
                dff1 = pd.DataFrame(ls, columns = ['tweet','view'])
                dff = pd.concat([dff1,dff])
                break
        clf.pipe(text)  # classify

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
dff = dff.reset_index(drop=True)
dff.tail(25)

In [ ]:
c = 0
for k in range(dff.shape[0]):
    lab = clf.pipe(dff['tweet'].iloc[k])[0]['label']
    if lab == dff['view'].iloc[k]:
        c = c+1



In [ ]:
c/dff.shape[0]

0.895924308588064

In [ ]:
dff.tail(50)

In [ ]:
df_hold.tail(50)

In [ ]:
df_hold.head(30)

In [ ]:
c = 0
for k in range(df_hold.shape[0]):
    lab = clf.pipe(df_hold['tweet'].iloc[k])[0]['label']
    if lab == df_hold['view'].iloc[k]:
        c = c+1


In [ ]:
c/df_hold.shape[0]

0.981675392670157

##Results:
~ 88% On Classifier
~ 90% On Generation